In [ ]:
!pip install openai==0.28
!pip install pandas
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## BOOLQ

import openai
import pandas as pd

openai.api_key = ""

# Load the CSV file
csv_file_path = '/content/drive/MyDrive/Algoverse/Old Results/AAVE Translations/GPT 4.0 Translations, 4.0 Evaluations/BoolQ/BoolQA_superglue_1000_4-0.csv'
data = pd.read_csv(csv_file_path)

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

#Translation API Call + Prompt

def translate_to_aave(sae_text, few_shot_examples):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following sentence from Standard English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Text: {sae_text}\n\n"
            "AAVE Translation:"
        )}
    ]

# Model
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# What columns are being made in what order
translated_data = pd.DataFrame(columns=['SAE Passage', 'SAE Question', 'AAVE Passage', 'AAVE Question'])

for index, row in data.iterrows():
    sae_passage = row['se passage']
    sae_question = row['se question']

    aave_passage = translate_to_aave(sae_passage, few_shot_examples)
    print(f"Processed row {index + 1} passage")

    aave_question = translate_to_aave(sae_question, few_shot_examples)
    print(f"Processed row {index + 1} question")
    # Creating new dataframe from results for csv file
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'SAE Passage': [sae_passage],
        'SAE Question': [sae_question],
        'AAVE Passage': [aave_passage],
        'AAVE Question': [aave_question]
    })], ignore_index=True)

#Output Translation Path
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated BoolQ.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

print(f"Translations completed and saved to {translated_csv_file_path}")

In [ ]:
## COPA

import openai
import pandas as pd

# Set up your OpenAI API key
openai.api_key = ""  # Replace with your actual OpenAI API key

# Load the CSV file for COPA
csv_file_path = '/content/drive/MyDrive/Algoverse/Old Results/AAVE Translations/GPT 4.0 Translations, 4.0 Evaluations/Copa/Copa_superglue_500_4.0.csv'
data = pd.read_csv(csv_file_path)

# Load the CSV file for Actual Answer
actual_answer_csv_path = '/content/drive/MyDrive/Algoverse/New Results/Evaluation Results/GPT-4o Eval Results/COPA/COPA_Evaluation_GPT4o.csv'
actual_answer_data = pd.read_csv(actual_answer_csv_path)

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

# Translation API Call + Prompt
def translate_to_aave(sae_text, few_shot_examples):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following sentence from Standard English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Text: {sae_text}\n\n"
            "AAVE Translation:"
        )}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Ensure the correct model is used
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Create a new DataFrame to store translations
translated_data = pd.DataFrame(columns=[
    'Premise', 'Choice 1', 'Choice 2',
    'Translated Premise', 'Translated Choice 1', 'Translated Choice 2', 'Actual Answer'
])

for index, row in data.iterrows():
    sae_premise = row['Premise']
    sae_choice1 = row['Choice1']
    sae_choice2 = row['Choice2']
    actual_answer = actual_answer_data.iloc[index]['Actual Answer']

    aave_premise = translate_to_aave(sae_premise, few_shot_examples)
    print(f"Processed row {index + 1} premise")

    aave_choice1 = translate_to_aave(sae_choice1, few_shot_examples)
    print(f"Processed row {index + 1} choice 1")

    aave_choice2 = translate_to_aave(sae_choice2, few_shot_examples)
    print(f"Processed row {index + 1} choice 2")

    # Creating new dataframe from results for csv file
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'Premise': [sae_premise],
        'Choice 1': [sae_choice1],
        'Choice 2': [sae_choice2],
        'Translated Premise': [aave_premise],
        'Translated Choice 1': [aave_choice1],
        'Translated Choice 2': [aave_choice2],
        'Actual Answer': [actual_answer]
    })], ignore_index=True)

# Save translated results to CSV file
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated Copa.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

print(f"Translations completed and saved to {translated_csv_file_path}")

In [ ]:
## MultiRC

import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual OpenAI API key

# Load the CSV file for MultiRC
csv_file_path = '/content/drive/MyDrive/Algoverse/Old Results/AAVE Translations/GPT 4.0 Translations, 4.0 Evaluations/Multi-RC/MultiRC_1000_4.0.csv'
data = pd.read_csv(csv_file_path)

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

#Translation API Call + Prompt

def translate_to_aave(sae_text, few_shot_examples):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following sentence from Standard English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Text: {sae_text}\n\n"
            "AAVE Translation:"
        )}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Ensure the correct model is used
        messages=messages,
        max_tokens=200,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Create a new DataFrame to store translations
translated_data = pd.DataFrame(columns=[
    'Paragraph', 'Question', 'Answer Choice',
    'Translated Paragraph', 'Translated Question', 'Translated Answer Choice',
    'Actual Label'
])

# Process each row for translation
for index, row in data.iterrows():
    sae_paragraph = row['Paragraph']
    sae_question = row['Question']
    sae_answer = row['Answer']
    actual_label = row['Actual Label']

    # Translate each part to AAVE
    aave_paragraph = translate_to_aave(sae_paragraph, few_shot_examples)
    print(f"Processed row {index + 1} paragraph")

    aave_question = translate_to_aave(sae_question, few_shot_examples)
    print(f"Processed row {index + 1} question")

    aave_answer = translate_to_aave(sae_answer, few_shot_examples)
    print(f"Processed row {index + 1} answer")

    # Creating new dataframe from results for csv file
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'Paragraph': [sae_paragraph],
        'Question': [sae_question],
        'Answer Choice': [sae_answer],
        'Translated Paragraph': [aave_paragraph],
        'Translated Question': [aave_question],
        'Translated Answer Choice': [aave_answer],
        'Actual Label': [actual_label]
    })], ignore_index=True)

# Save translated results to CSV file
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated MultiRC.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

# Print function
print(f"Translations completed and saved to {translated_csv_file_path}")

In [ ]:
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual OpenAI API key

# Load the CSV file for SST-2
csv_file_path = '/content/drive/MyDrive/Algoverse/Old Results/AAVE Translations/GPT 4.0 Translations, 4.0 Evaluations/SST-2/SST-2_super-glue_1000_4.0.csv'
data = pd.read_csv(csv_file_path)

# Load the CSV file for Actual Label
actual_label_csv_path = '/content/drive/MyDrive/Algoverse/New Results/Evaluation Results/GPT-4-turbo Eval Results/SST-2/SST-2_Evaluation_GPT-4o-Mini.csv'
actual_label_data = pd.read_csv(actual_label_csv_path)

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

# Translation API Call + Prompt
def translate_to_aave(sae_text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following sentence from Standard English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Text: {sae_text}\n\n"
            "AAVE Translation:"
        )}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Ensure the correct model is used
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Create a new DataFrame to store translations
translated_data = pd.DataFrame(columns=[
    'Original Sentence', 'Translated Sentence', 'Actual Label'
])

# Process each row for translation
for index, row in data.iterrows():
    original_sentence = row['Original']
    actual_label = actual_label_data.iloc[index]['Actual Label']

    # Translate each part to AAVE
    translated_sentence = translate_to_aave(original_sentence)
    print(f"Processed row {index + 1}")

    # Creating new dataframe from results for csv file
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'Original Sentence': [original_sentence],
        'Translated Sentence': [translated_sentence],
        'Actual Label': [actual_label]
    })], ignore_index=True)

# Save translated results to CSV file
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated SST-2.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

print(f"Translations completed and saved to {translated_csv_file_path}")

In [ ]:
## Cola Translations

import openai
import pandas as pd
from datasets import load_dataset

# Set your OpenAI API key
openai.api_key = ""  # Replace with your actual OpenAI API key

# Load the CoLA subset from the GLUE dataset
dataset = load_dataset("glue", "cola")

# Select the 'train' split and take the first 1000 entries
data = dataset['train'].select(range(1000))

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

#Translation API Call + Prompt

def translate_to_aave(sae_text, few_shot_examples):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following sentence from Standard English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Text: {sae_text}\n\n"
            "AAVE Translation:"
        )}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Ensure the correct model is used
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Create a DataFrame to store translations
translated_data = pd.DataFrame(columns=[
    'Sentence', 'Translated Sentence', 'Original Label'
])

# Process each row for translation
for index, row in enumerate(data):
    original_sentence = row['sentence']
    original_label = row['label']

    # Translate the sentence to AAVE
    translated_sentence = translate_to_aave(original_sentence, few_shot_examples)
    print(f"Processed row {index + 1}")

    # Add the results to the DataFrame
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'Sentence': [original_sentence],
        'Translated Sentence': [translated_sentence],
        'Original Label': [original_label]
    })], ignore_index=True)

# Save translated results to CSV file
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated CoLA.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

print(f"Translations completed and saved to {translated_csv_file_path}")

In [ ]:
## WSC Translations

import openai
import pandas as pd

# Set up your OpenAI API key
openai.api_key = ""

# Load the combined CSV file
csv_file_path = '/content/drive/MyDrive/wsc_combined_translated.csv'
data = pd.read_csv(csv_file_path)

# Few-shot examples for translation to AAVE
few_shot_examples = [
    "I was bewildered, but I knew dat it was no gud asking his ass to explain.",
    "Cochran pontificated windily for da camera.",
    "I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go."
]

# Translation API Call + Prompt
def translate_to_aave(paragraph, span1_text, span2_text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that translates Standard American English (SAE) to African American Vernacular English (AAVE)."},
        {"role": "user", "content": (
            "Translate the following passage from Standard American English to African American Vernacular English (AAVE). "
            "Ensure the translation maintains the structure of the original sentence without adding extra information.\n\n"
            "Examples for reference:\n"
            "1. I was bewildered, but I knew dat it was no gud asking his ass to explain.\n"
            "2. Cochran pontificated windily for da camera.\n"
            "3. I don’t want them to follow in my footsteps, as I ain’t go to no college, but I want them to go.\n\n"
            f"Make sure to include the phrases '{span1_text}' and '{span2_text}' exactly as they appear in the text.\n\n"
            f"Passage: {paragraph}\n\n"
            "AAVE Translation:"
        )}
    ]

    # Model
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    return response.choices[0].message['content'].strip()

# Define the columns for the output DataFrame
translated_data = pd.DataFrame(columns=['Original Paragraph', 'AAVE Paragraph', 'Span 1', 'Span 2', 'Actual Label'])

for index, row in data.iterrows():
    original_paragraph = row['Original Paragraph']
    span1_text = row['Span 1']
    span2_text = row['Span 2']
    actual_label = row['Actual Label']

    # Translate the paragraph to AAVE
    aave_paragraph = translate_to_aave(original_paragraph, span1_text, span2_text)
    print(f"Processed row {index + 1}")

    # Append the results to the DataFrame
    translated_data = pd.concat([translated_data, pd.DataFrame({
        'Original Paragraph': [original_paragraph],
        'AAVE Paragraph': [aave_paragraph],
        'Span 1': [span1_text],
        'Span 2': [span2_text],
        'Actual Label': [actual_label]
    })], ignore_index=True)

# Output Translation Path
translated_csv_file_path = '/content/drive/MyDrive/Algoverse/New Results/GPT 4o mini Translations/Translated WSC.csv'
translated_data.to_csv(translated_csv_file_path, index=False)

print(f"Translations completed and saved to {translated_csv_file_path}")